___
<h3 style="text-align:center;">⚠️ Entrenamiento de <u>modelos</u> ⚠️
</h3>

---

<h3 style="text-align:center;">📚🔧
</h3>

In [ ]:
# Importacion de Librerías:
import pandas as pd
import numpy as np

from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

import pickle
import yaml

---
**<h3 style="text-align:center;">📚🔧: División: <u>X_train</u> , <u>y_train</u> , <u>X_test</u> , <u>y_test</u></h3>**

**X_train + y_train:**

In [24]:
df_train = pd.read_csv('../data/train.csv', index_col='Unnamed: 0')

X_train = df_train.drop('Diagnosis', axis=1)
y_train = df_train['Diagnosis']

**X_test + y_test:**

In [25]:
df_test = pd.read_csv('../data/test.csv', index_col='Unnamed: 0')

X_test = df_test.drop('Diagnosis', axis=1)
y_test = df_test['Diagnosis']

**☑️ Check Dimensiones:**

In [21]:
# Forma de variables de entrenamiento y comevaluación:
print('Forma de X_train:', X_train.shape)
print('Forma de y_train:', y_train.shape)
print('Forma de X_test:', X_test.shape)
print('Forma de y_test:', y_test.shape)

Forma de X_train: (1125, 10)
Forma de y_train: (1125,)
Forma de X_test: (375, 10)
Forma de y_test: (375,)


---
**<h3 style="text-align:center;">🤖¹ : <u>DecisionTreeClassifier</u>🌳 +  <u>RandomizedSearchCV</u>🔄</h3>**

**🏋️‍♂️ Entrenamiento:**

In [27]:
modelo_dtc = DecisionTreeClassifier()


params_dtc = {
    'max_depth': np.arange(1, 20),
    'min_samples_split': np.arange(2, 20),
    'min_samples_leaf': np.arange(1, 20),
    'criterion': ['gini', 'entropy']
}

rs_dtc = RandomizedSearchCV(
    modelo_dtc, param_distributions=params_dtc,
    n_iter=50, scoring='accuracy', cv=5, random_state=42, n_jobs=-1
)

rs_dtc.fit(X_train, y_train)

print("Mejores parámetros:", rs_dtc.best_params_)
print("Mejor puntuación de validación:", rs_dtc.best_score_)

Mejores parámetros: {'min_samples_split': np.int64(6), 'min_samples_leaf': np.int64(3), 'max_depth': np.int64(9), 'criterion': 'entropy'}
Mejor puntuación de validación: 0.8728888888888889


c:\Python312\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


**🔮 Predicciones:**

In [ ]:
best_dtc = rs_dtc.best_estimator_

pred_dtc = best_dtc.predict(X_test)

df_predicciones_dtc = pd.DataFrame({'y_real': y_test,  
                                'y_pred': pred_dtc})

df_predicciones_dtc

,y_real,y_pred
1116,1,1
1368,1,1
422,0,0
413,0,0
451,0,0
...,...,...
155,1,1
1151,0,0
1141,0,0
974,0,1


**✔️ Evaluación:**

In [ ]:
print('accuracy:',accuracy_score(y_test, pred_dtc))
print('recall:',recall_score(y_test,pred_dtc))
print('precission:', precision_score(y_test, pred_dtc))

print('\n----------------------------------------------\n')

print('confusion_matrix:\n',confusion_matrix(y_test, pred_dtc))

accuracy: 0.8853333333333333
recall: 0.823943661971831
precission: 0.8666666666666667

----------------------------------------------

confusion_matrix:
 [[215  18]
 [ 25 117]]


In [ ]:
print(classification_report(y_test, pred_dtc))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       233
           1       0.87      0.82      0.84       142

    accuracy                           0.89       375
   macro avg       0.88      0.87      0.88       375
weighted avg       0.88      0.89      0.88       375



**💾 Guardar Modelo + Config:**

In [45]:
with open('../models/DecissionTree_RandomSearchCV/model_DTCRSCV.pkl', 'wb') as archivo:
    pickle.dump(best_dtc, archivo)

In [46]:
configuracion = {
    'modelo': 'DecisionTreeClassifier',
    'mejores_parametros': rs_dtc.best_params_,
    'mejor_puntuacion': rs_dtc.best_score_,
    'configuracion_busqueda': {
        'n_iter': 50,
        'scoring': 'accuracy',
        'cv': 5,
        'random_state': 42
    }
}

with open('../models/DecissionTree_RandomSearchCV/config_DTCRSCV.yaml', 'w') as archivo:
    yaml.dump(configuracion, archivo)

---
**<h3 style="text-align:center;">🤖² : <u>RandomForestClassifier</u>🌳 +  <u>RandomizedSearchCV</u>🔄</h3>**

**🏋️‍♂️ Entrenamiento:**

**🔮 Predicciones:**

**✔️ Evaluación:**

**💾 Guardar Modelo + Config:**

---
**<h3 style="text-align:center;">🤖³ :</h3>**

**🏋️‍♂️ Entrenamiento:**

**🔮 Predicciones:**

**✔️ Evaluación:**

**💾 Guardar Modelo + Config:**

---
**<h3 style="text-align:center;">🤖⁴ :</h3>**

**🏋️‍♂️ Entrenamiento:**

**🔮 Predicciones:**

**✔️ Evaluación:**

**💾 Guardar Modelo + Config:**

---
**<h3 style="text-align:center;">🤖⁵ :</h3>**

**🏋️‍♂️ Entrenamiento:**

**🔮 Predicciones:**


**✔️ Evaluación:**


**💾 Guardar Modelo + Config:**
